In [7]:
import gym
import random
import numpy as np

random.seed(1234)
streets = gym.make('Taxi-v3', render_mode='ansi').env
streets.reset()
print("\n" + streets.render())


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




- R, G, B e Y são pontos de início e fim
- A letra __AZUL__ indica o local onde deve-se buscar o passageiro
- A letra __MAGENTA__ indica o destino do passageiro
- As linhas sólidas representam paredes
- O retângulo representa o táxi (amarelo quando vazio e verde quando carregando um passageiro)
- Em cada estado há 6 possibilidades: mover para norte, sul, leste e oeste, pegar um passageiro e deixar um passageiro
- Deixar um passageiro no local correto vale +20
- Cada passo dado com um passageiro vale -1
- Pegar um passageiro ou deixar no local ilegal vale -10

In [2]:
# estado inicial
initial_state = streets.encode(2, 3, 2, 0) # (linha, coluna, passageiro, destino)
streets.s = initial_state
print("\n" + streets.render())


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [3]:
# tabela de recompensas
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

In [4]:
# cria a q-table
q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

# define os parametros
learning_rate = 0.1
discount_factor = 0.6
exploration = 0.1
epochs = 10000

# treinamento
for taxi_run in range(epochs):
    state = streets.reset()
    # se o estado for uma tupla, pega o primeiro elemento
    if isinstance(state, tuple):
        state = state[0] 
    
    done = False
    
    while not done:
        random_value = random.uniform(0, 1)
        if (random_value < exploration):
            action = streets.action_space.sample()
        else:
            action = np.argmax(q_table[state])
            
        next_state, reward, done, info, truncated = streets.step(action)
        
        # se o proximo estado for uma tupla, pega o primeiro elemento
        if isinstance(next_state, tuple):
            next_state = next_state[0]
            
        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q) # q-learning formula
        q_table[state, action] = new_q
        
        state = next_state

c:\Users\kaiqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
q_table[initial_state]

array([-2.41254986, -2.40988509, -2.39574863, -2.3639511 , -4.5035904 ,
       -8.99500564])